In [1]:
# Dependiencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time

In [2]:
# Prepare chromedriver
# MUST USE CHROME 79
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Mars News Title and Text

In [3]:
# Retrieve page
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

# Wait for page load
time.sleep(2)

In [4]:
# Create soup object and parse first news article title/text
news_soup = bs(browser.html, 'html.parser')

news_title = news_soup.find("div",class_="content_title").text.strip()
news_text = news_soup.find("div",class_="rollover_description").text.strip()

print(f'''
    Title: {news_title}
    Text: {news_text}
''')


    Title: NASA's Mars 2020 Rover Completes Its First Drive
    Text: In a 10-plus-hour marathon, the rover steered, turned and drove in 3-foot (1-meter) increments over small ramps.



## Mars Image

In [5]:
# Retrieve page
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [6]:
# Create soup object
img_soup = bs(browser.html, 'html.parser')

# Get popup link
img_page = 'https://www.jpl.nasa.gov' + str(img_soup.find('li', class_='slide').find('a')['data-link'])

In [7]:
# Create soup object of popup page
browser.visit(img_page)
img_large_soup = bs(browser.html, 'html.parser')

# Get large image link
img_large = 'https://www.jpl.nasa.gov' + str(img_large_soup.find('figure').find('a')['href'])

browser.quit()

print(img_large)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23599_hires.jpg


## Mars Weather

In [8]:
# Retrieve page
weather_url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(weather_url)

In [9]:
# Crete soup object and parse weather paragraph
soup = bs(response.text, 'html.parser')

mars_weather = soup.find('div', class_="js-tweet-text-container").find('p').text.split('hPapic')[0].rstrip()
        

print(mars_weather)

InSight sol 378 (2019-12-19) low -97.7ºC (-143.8ºF) high -20.0ºC (-3.9ºF)
winds from the SSE at 5.7 m/s (12.8 mph) gusting to 23.8 m/s (53.2 mph)
pressure at 6.50


## Mars Facts

In [11]:
# Retrieve page into pandas
facts_url = 'https://space-facts.com/mars/'
mars_facts_df = pd.read_html(facts_url)[0]

In [12]:
# Remove column names
mars_facts_df.columns = ['','']

mars_facts_df

,,
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
mars_facts_df.to_html('mars_facts.html')

## Mars Hemispheres

In [14]:
# Retrieve mars hemispheres page and create soup object
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(hemispheres_url)
soup = bs(response.text, 'html.parser')

# Get url for each hemisphere page
hemispheres_url_list = []

for x in range(len(soup.find_all('div', class_='item'))):
    hemispheres_url_list.append('https://astrogeology.usgs.gov' + soup.find_all('div', class_='item')[x].find('a')['href'])

# Create a list of dictionaries containing the title and img url for each hemisphere
hemisphere_image_list = []

for url in range(len(hemispheres_url_list)):
    
    hemisphere_image_dict = {}
    
    response = requests.get(hemispheres_url_list[url])
    soup = bs(response.text, 'html.parser')
    
    title = soup.find('title').text.split('Enhanced')[0].rstrip()
    img_url = soup.find('div', class_='downloads').find('a')['href']
    
    hemisphere_image_dict['title'] = title
    hemisphere_image_dict ['img_url'] = img_url
    
    hemisphere_image_list.append(hemisphere_image_dict)
    
print(hemisphere_image_list)

[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
